In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler


In [22]:
# Загрузка данных
data = pd.read_csv('/home/eva/project/predictive_real_estate_price_analytics_service/data/progress/progress_data.csv')
data.sort_values(by='total_meters')

,url_id,total_meters,price
496,307723781,10.7,3200000
672,279298831,13.3,4287787
713,317285359,17.0,7890000
709,304863760,17.2,6141432
505,313373364,18.6,4270000
...,...,...,...
145,316935589,117.5,69500000
301,243257002,118.0,38700000
406,316491879,118.0,57700000
309,306278529,118.4,25900000


In [21]:

# Стандартизация признака
#scaler = StandardScaler()
#data['total_meters'] = scaler.fit_transform(data[['total_meters']])

# Разделение на признаки и целевую переменную
X = data[['total_meters']]  # только один признак - площадь
y = data['price']
y_log = np.log1p(y)
y = y_log
# Разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Создание и обучение модели
model = LinearRegression()
print(X_train)
print(y_train)
model.fit(X_train, y_train)


# Предсказание и обратное преобразование из логарифма
y_pred_log = model.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_test_orig = np.expm1(y_test)

# Оценка модели
mse = mean_squared_error(y_test_orig, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_orig, y_pred)
mae = np.mean(np.abs(y_test_orig - y_pred))

# Вывод метрик
# Вывод метрик качества
# todo: use logging
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.2f}")
print(f"Коэффициент детерминации R²: {r2:.6f}")
print(f"Средняя ошибка предсказания: {np.mean(np.abs(y_test - y_pred)):.2f} рублей")


'''
# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Вывод метрик качества
# todo: use logging
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.2f}")
print(f"Коэффициент детерминации R²: {r2:.6f}")
print(f"Средняя ошибка предсказания: {np.mean(np.abs(y_test - y_pred)):.2f} рублей")

# Коэффициенты модели
print(f"Коэффициент при площади: {model.coef_[0]:.2f}")
print(f"Свободный член: {model.intercept_:.2f}")

'''



'''

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np


X = data[['total_meters']]  # только один признак - площадь
y = data['price']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Создание и обучение модели CatBoost
model = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=6,
    verbose=100,  # выводит информацию каждые 100 итераций
    random_state=42
)
model.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Оценка модели
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Вывод метрик качества
print(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")
print(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.2f}")
print(f"Коэффициент детерминации R²: {r2:.6f}")
print(f"Средняя ошибка предсказания: {np.mean(np.abs(y_test - y_pred)):.2f} рублей")

# Для CatBoost коэффициенты модели интерпретируются иначе, чем в линейной регрессии
# Можно получить важность признаков
print("\nВажность признаков:")
for name, score in zip(X.columns, model.get_feature_importance()):
    print(f"{name}: {score:.2f}")
'''

      total_meters
982           43.0
969           61.0
10            80.1
147           82.6
346           66.0
...            ...
1095          43.0
1130          69.0
1294          42.8
860           45.4
1126          85.9

[1043 rows x 1 columns]
982     16.299584
969     17.249498
10      18.112796
147     17.837284
346     17.092655
          ...    
1095    17.385043
1130    17.665658
1294    16.341239
860     15.931766
1126    17.599700
Name: price, Length: 1043, dtype: float64
Среднеквадратичная ошибка (MSE): 310782163398415.31
Корень из среднеквадратичной ошибки (RMSE): 17629014.82
Коэффициент детерминации R²: 0.371199
Средняя ошибка предсказания: 28341521.09 рублей


'\n\nfrom catboost import CatBoostRegressor\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.metrics import mean_squared_error, r2_score\nimport numpy as np\n\n\nX = data[[\'total_meters\']]  # только один признак - площадь\ny = data[\'price\']\n\n\nX_train, X_test, y_train, y_test = train_test_split(\n    X, y, test_size=0.2, random_state=42)\n\n# Создание и обучение модели CatBoost\nmodel = CatBoostRegressor(\n    iterations=1000,\n    learning_rate=0.05,\n    depth=6,\n    verbose=100,  # выводит информацию каждые 100 итераций\n    random_state=42\n)\nmodel.fit(X_train, y_train)\n\n# Предсказание на тестовой выборке\ny_pred = model.predict(X_test)\n\n# Оценка модели\nmse = mean_squared_error(y_test, y_pred)\nrmse = np.sqrt(mse)\nr2 = r2_score(y_test, y_pred)\n\n# Вывод метрик качества\nprint(f"Среднеквадратичная ошибка (MSE): {mse:.2f}")\nprint(f"Корень из среднеквадратичной ошибки (RMSE): {rmse:.2f}")\nprint(f"Коэффициент детерминации R²: {r2:.6f}")\nprint(f"Сред

In [ ]:
import joblib

# Сохранение модели
model_path = '/home/eva/project/predictive_real_estate_price_analytics_service/models/linear_regression_model.pkl'

def model_pipline
joblib.dump(model, model_path)
print(f"Модель сохранена в файл {model_path}") 

# Загрузка модели
loaded_model = joblib.load(model_path)
print("Модель загружена из файла")

Модель сохранена в файл /home/eva/project/predictive_real_estate_price_analytics_service/models/linear_regression_model.pkl
Модель загружена из файла
